# 유니코드 문자열

자연어 처리 모델(NLP models)는 종종 다른 문자들의 세트로 이루어진 다양한 언어를 다루게 됩니다. 유니코드는 표준 인코딩 시스템으로, 거의 모든 언어의 문자들을 나타내는데 사용됩니다.

모든 유니코드 문자는 고유의 정수 code point(0~0x10FFFF)로 인코딩됩니다. 유니코드 문자열은 0 또는 코드 포인트들의 시퀀스입니다.

이번 튜토리얼에서는 텐서플로우에서 어떻게 유니코드 문자열을 나타내는지 보여주고, 표준 문자열과 대응시켜주는 유니코드 대응기(Unicode Equivalent)를 이용해 유니코드 문자열을 어떻게 조작하는지 보여줍니다. 유니코드 문자열을 토큰으로 분리시키는 방법을 이용합니다.

In [1]:
import tensorflow as tf
import numpy as np

## 1. `tf.string` 데이터 타입

텐서플로의 기본 `tf.string` 데이터 타입(dtype)은 byte string로 이루어진 텐서를 만듭니다. 유니코드 문자열은 기본적으로 utf-8로 인코딩 됩니다.

In [2]:
tf.constant(u"Thanks 😊")

<tf.Tensor: shape=(), dtype=string, numpy=b'Thanks \xf0\x9f\x98\x8a'>

`tf.string` 텐서는 byte string을 최소 단위로 다루기 때문에 다양한 길이의 byte string을 저장할 수 있습니다.

문자열의 길이는 텐서 차원(dimensions)에 포함되지 않습니다.

In [3]:
tf.constant([u"You're", u"welcome!"]).shape

TensorShape([2])

Note : 파이썬을 사용해 문자열을 만들 때 버전 2와 버전 3에서 유니코드를 다루는 방식이 다릅니다. 버전 2에서는 위와 같이 "u"접두사를 사용하여 유니코드 문자열을 나타냅니다. 버전 3에서는 유니코드 인코딩된 문자열이 기본값입니다.

## 2. 유니코드 표현

텐서플로에서는 유니코드 문자열을 표현하기 위한 두 가지 방법이 있습니다.
+ `string` scalar - code point 시퀀스가 이미 알려진 문자 인코딩법을 사용해 인코딩됩니다.
+ `int32` vector - 각 위치가 하나의 code point를 가집니다.

예를 들어, 아래의 세 가지 값이 유니코드 문자열인 "语言处理"을 나타냅니다.

In [4]:
# string scalar 방법으로 인코딩된 UTF-8
text_utf8 = tf.constant(u"语言处理")
text_utf8

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe8\xaf\xad\xe8\xa8\x80\xe5\xa4\x84\xe7\x90\x86'>

In [5]:
# string scalar 방법으로 인코딩된 UTF-16-BE
text_utf16be = tf.constant(u"语言处理".encode("UTF-16-BE"))
text_utf16be

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8b\xed\x8a\x00Y\x04t\x06'>

In [6]:
# int32 vector 방법으로 인코딩된 유니코드 코드 포인트 벡터
text_chars = tf.constant([ord(char) for char in u"语言处理"])
text_chars

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35821, 35328, 22788, 29702], dtype=int32)>

### 하나의 유니코드 표현법에서 다른 표현법으로 변환하기

+ `tf.strings.unicode_decode`: string scalar를 vector code point로 바꿉니다.
+ `tf.strings.unicode_encode`: vector code point를 string scalar로 바꿉니다.
+ `tf.strings.unicode_transcode`: string scalar를 다른 string scalar로 바꿉니다.

In [7]:
tf.strings.unicode_decode(text_utf8, input_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35821, 35328, 22788, 29702], dtype=int32)>

In [8]:
tf.strings.unicode_encode(text_chars, output_encoding='UTF-8')

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe8\xaf\xad\xe8\xa8\x80\xe5\xa4\x84\xe7\x90\x86'>

In [9]:
tf.strings.unicode_transcode(text_utf8, input_encoding='UTF8', output_encoding='UTF-16-BE')

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8b\xed\x8a\x00Y\x04t\x06'>

### 차원을 배치시키기

여러 문자열을 디코딩할 때 각 문자열의 문자의 개수가 같지 않을 것입니다. 이러한 결괏값을 `tf.RaggedTensor`라고 합니다.

In [10]:
batch_utf8 = [s.encode('UTF-8') for s in
              [u'hÃllo', u'What is the weather tomorrow', u'Göödnight', u'😊']]
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8, input_encoding='UTF-8')
for sentence_chars in batch_chars_ragged.to_list():
    print(sentence_chars)

[104, 195, 108, 108, 111]
[87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116, 104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]
[71, 246, 246, 100, 110, 105, 103, 104, 116]
[128522]


이러한 `tf.RaggedTensor`를 바로 사용할 수도 있고, 패딩을 거친 `tf.Tensor`로 변환하거나 `tf.SparseTensor`로 변환할 수 있습니다. 각각으로 변환할 때는 `tf.RaggedTensor.to_tensor`와 `tf.RaggedTensor.to_sparse`를 이용합니다.

In [11]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

[[   104    195    108    108    111     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [    87    104     97    116     32    105    115     32    116    104
     101     32    119    101     97    116    104    101    114     32
     116    111    109    111    114    114    111    119]
 [    71    246    246    100    110    105    103    104    116     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [128522     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]]


In [12]:
batch_chars_sparse = batch_chars_ragged.to_sparse()

nrows, ncols = batch_chars_sparse.dense_shape.numpy()
elements = [['_' for i in range(ncols)] for j in range(nrows)]
for (row, col), value in zip(batch_chars_sparse.indices.numpy(), batch_chars_sparse.values.numpy()):
    elements[row][col] = str(value)
value_lengths = []
for row in elements:
    for value in row:
        value_lengths.append(len(value))
max_width = max(value_lengths)
print('[%s]' % '\n '.join(
    '[%s]' % ', '.join(value.rjust(max_width) for value in row)
    for row in elements
))


[[   104,    195,    108,    108,    111,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _]
 [    87,    104,     97,    116,     32,    105,    115,     32,    116,    104,    101,     32,    119,    101,     97,    116,    104,    101,    114,     32,    116,    111,    109,    111,    114,    114,    111,    119]
 [    71,    246,    246,    100,    110,    105,    103,    104,    116,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _]
 [128522,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _,      _]]


같은 길이의 여러 문자열을 인코딩할 때 `tf.Tensor`를 인풋으로 이용합니다.

In [13]:
tf.strings.unicode_encode([[99, 97, 116], [100, 111, 103], [99, 111, 119]],
                          output_encoding='UTF-8')

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'cat', b'dog', b'cow'], dtype=object)>

다른 길이의 여러 문자열을 인코딩할 때는 `tf.RaggedTensor`를 input으로 이용합니다.

In [14]:
tf.strings.unicode_encode(batch_chars_ragged, output_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

만약 여러 문자열이 스페어스(sparse)되었거나 패딩(padding)된 포맷이라면 먼저 `tf.RaggedTensor`로 변환합니다.

In [15]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8'
)

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

In [16]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1),
    output_encoding='UTF-8'
)

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

### 3. 유니코드 연산

### 문자 길이

`tf.strings.length`는 기본적으로 인코딩된 문자열의 바이트 크기를 알려줍니다. `"UTF8_CHAR"` 또는 `"UTF16_CHAR"`을 지정해주어 인코딩된 문자열의 UTF-8 또는 UTF-16 문자 길이를 알 수 있습니다.

In [17]:
# 마지막 이모티콘은 4바이트 크기이다.
thanks = u'Thanks 😊'.encode('UTF-8')
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} bytes; {} UTF-8 characters'.format(num_bytes, num_chars))

11 bytes; 8 UTF-8 characters


### 부분 문자열

`tf.strings.substr`은 `unit`매개변수 값을 사용해 `pos`와 `len` 매개변수로 지정된 문자열의 종류를 결정합니다.

In [19]:
# len=1이면 바이트 하나를 반환합니다.
tf.strings.substr(thanks, pos=7, len=1).numpy()

b'\xf0'

In [20]:
# unit='UTF8_CHAR'로 지정하면 4 바이트인 문자 하나를 반환합니다.
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

b'\xf0\x9f\x98\x8a'


### 유니코드 문자열 분리

`tf.strings.unicode_split`은 유니코드 문자열을 개별 문자로 분리합니다.

In [21]:
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

array([b'T', b'h', b'a', b'n', b'k', b's', b' ', b'\xf0\x9f\x98\x8a'],
      dtype=object)

### 문자 바이트 오프셋

각 문자까지의 바이트 크기와 codepoint를 알고 싶다면 `tf.strings.unicode_decode_with_offsets`를 이용할 수 있습니다.

In [22]:
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u'🎈🎉🎊', 'UTF-8')

for codepoint, offset in zip(codepoints.numpy(), offsets.numpy()):
    print('At byte offset {}: codepoint {}'.format(offset, codepoint))

At byte offset 0: codepoint 127880
At byte offset 4: codepoint 127881
At byte offset 8: codepoint 127882


## 4. 유니코드 스크립트

`script`는 codepoint의 컬렉션을 의미합니다. 각 유니코드 codepoint는 하나의 script에 속에 있습니다. 예를 들어 'Б'라는 문자는 러시아와 우크라이나와 같은 Slavic 언어로부터 파생된 문자로 Cyrillic script에 속해있습니다.

`tf.strings.unicode_script`를 이용하면 각 유니코드 codepoint가 속에 있는 script를 `int32`의 형태로 알 수 있습니다.

In [23]:
uscript = tf.strings.unicode_script([33464, 1041]) # ['芸', 'Б']

print(uscript.numpy()) # [17, 8] == [USCRIPT_HAN, USCRIPT_CYRILLIC]

[17  8]


`tf.strings.unicode_script`는 `tf.Tensor` 또는 `tf.RaggedTensor` 형태에도 사용할 수 있습니다.

In [24]:
print(tf.strings.unicode_script(batch_chars_ragged))

<tf.RaggedTensor [[25, 25, 25, 25, 25], [25, 25, 25, 25, 0, 25, 25, 0, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25, 25], [25, 25, 25, 25, 25, 25, 25, 25, 25], [0]]>


## 5. 예제 : 간단한 분할

분할(segmentation)은 텍스트를 단어와 같은 단위로 나누는 작업입니다. 공백 문자가 단어를 나누는 구분자로 사용되는 경우는 간단하지만, (중국어나 일본어와 같이) 공백을 사용하지 않는 언어나 (독일어 같이) 단어를 길게 조합하는 언어는 의미를 분석하기 위한 분할 과정이 꼭 필요합니다. 

이번 분할 예제는 공백 문자는 모두 uscript_common(실제 텍스트의 스크립트 코드와 다른 특별한 스크립트 코드)로 분류하기 때문에 공백을 사용하는 대부분의 언어들에서도 적용됩니다.

In [25]:
# 처리할 문장들 입니다. 이 라인을 수정해서 다른 입력값을 시도해 보세요!
sentence_texts = [u'Hello, world.', u'世界こんにちは']

먼저 문장을 문자 codepoint로 디코딩하고 각 문자에 대한 스크립트 식별자를 찾습니다.

In [26]:
sentence_char_codepoint = tf.strings.unicode_decode(sentence_texts, 'UTF-8')
print(sentence_char_codepoint)

sentence_char_script = tf.strings.unicode_script(sentence_char_codepoint)
print(sentence_char_script)

<tf.RaggedTensor [[72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 46], [19990, 30028, 12371, 12435, 12395, 12385, 12399]]>
<tf.RaggedTensor [[25, 25, 25, 25, 25, 0, 0, 25, 25, 25, 25, 25, 0], [17, 17, 20, 20, 20, 20, 20]]>


스크립트 식별자를 이용해서 단어 경계가 추가될 위치를 결정합니다. 각 문장의 시작, 이전 문자와 스크립트가 다른 문자의 위치(시작 위치) 모두 단어 경계(True)로 변경합니다.

예를 들어 "世界こんにちは"가 있다면 7개의 인덱스 중 문장의 시작 위치인 1번 인덱스와 한자에서 일본어로 바뀌는 3번 위치를 True로 하고 나머지 인덱스는 모두 False로 합니다.
[True, False, True, False, False, False, False]
즉, 다른 언어로 바뀔 때 시작 인덱스를 True로 두어 다른 언어가 시작된다는 것을 알 수 있습니다.

In [37]:
# sentence_char_starts_word[i, j]는
# i번째 문장 안에 있는 j번째 문자가 단어의 시작이면 True 입니다.
sentence_char_starts_word = tf.concat(
    [tf.fill([sentence_char_script.nrows(), 1], True),
     tf.not_equal(sentence_char_script[:, 1:], sentence_char_script[:, :-1])],
     axis=1
)
print(sentence_char_starts_word)
# True가 나타난 인덱스를 뽑아냅니다.
word_starts = tf.squeeze(tf.where(sentence_char_starts_word.values), axis=1)
print(word_starts)

<tf.RaggedTensor [[True, False, False, False, False, True, False, True, False, False, False, False, True], [True, False, True, False, False, False, False]]>
tf.Tensor([ 0  5  7 12 13 15], shape=(6,), dtype=int64)


이 시작 인덱스를 사용해서 단어 또는 언어가 구분된 `RaggedTensor`를 만듭니다.

In [31]:
# 문장의 구분 없이 단어만을 일렬로 나열합니다.
word_char_codepoint = tf.RaggedTensor.from_row_starts(
    values=sentence_char_codepoint.values,
    row_starts=word_starts
)
print(word_char_codepoint)

<tf.RaggedTensor [[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46], [19990, 30028], [12371, 12435, 12395, 12385, 12399]]>


마지막으로 단어 코드 포인트 `RaggedTensor`를 문장으로 다시 나눕니다.

In [33]:
# 각 문장 안에 있는 단어의 수를 뽑아냅니다.
# True는 1, False는 0이므로 단어의 수만큼 True(1)가 있으므로 모두 더하면 됩니다.
sentence_num_words = tf.reduce_sum(
    tf.cast(sentence_char_starts_word, tf.int64),
    axis=1
)
print(sentence_num_words)

# 일렬로 나열된 단어들을 단어 수 별로 구분해서 묶으면 문장을 구분됩니다.
sentence_word_char_codepoint = tf.RaggedTensor.from_row_lengths(
    values=word_char_codepoint,
    row_lengths=sentence_num_words
)
print(sentence_word_char_codepoint)

tf.Tensor([4 2], shape=(2,), dtype=int64)
<tf.RaggedTensor [[[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46]], [[19990, 30028], [12371, 12435, 12395, 12385, 12399]]]>


최종 결과를 읽기 쉽게 utf-8 문자열로 다시 인코딩합니다.

In [34]:
tf.strings.unicode_encode(sentence_word_char_codepoint, 'UTF-8').to_list()

[[b'Hello', b', ', b'world', b'.'],
 [b'\xe4\xb8\x96\xe7\x95\x8c',
  b'\xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf']]